In [1]:
from spacy_dish_name import *
import pandas as pd

In [2]:
IS_TEST = True

In [3]:
file_path = '/Users/stephenzhang/Downloads/yelp.csv'
df = pd.read_csv(file_path)
if IS_TEST:
    sampled_business_ids = df['business_id'].sample(n=3, random_state=123).tolist()
    df = df[df['business_id'].isin(sampled_business_ids)]

In [4]:
df['text_clean'] = df['text'].apply(clean_text)

In [5]:
ACTION_VERBS = get_action_verbs(df, 'text_clean', 20)
MENUE_ITEMS = get_menu_items(df, 'text_clean', 20)

In [ ]:

def apply_spacy_to_df(
    df, action_verbs, menu_items, filter_validate_menu=False, min_freq=2
):
    # Extract raw dishes
    df["dishes_raw"] = df["text_clean"].apply(
        lambda text: extract_dishes_spacy(text, action_verbs)
    )

    # Lemmatize the extracted dishes
    df["dishes_lemmatized"] = df["dishes_raw"].apply(
        lambda dishes: [lemmatize_dish(dish) for dish in dishes]
    )
    # Filter dishes based on frequency
    all_texts = df["text_clean"].tolist()
    filtered_dishes = filter_dishes_spacy(all_texts, action_verbs, min_freq)

    # Add filtered dishes to the DataFrame
    df["filtered_dishes"] = df["dishes_lemmatized"].apply(
        lambda dishes: [dish for dish in dishes if dish in filtered_dishes]
    )

    # Optionally validate dishes against a menu
    if filter_validate_menu:
        df["valid_dishes"] = df["filtered_dishes"].apply(
            lambda dishes: [dish for dish in dishes if dish in menu_items]
        )

    return df

In [11]:
test_df = apply_spacy_to_df(df, ACTION_VERBS, MENUE_ITEMS)